                   START
                    |
                    |
                    |
                    |
                GENERATE_JOKE
                    |
                    |
                    |
                    |
                GENERATE EXPLANATION
                    |
                    |
                    |
                    |
                   END


In [6]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver # for Store in RAM
load_dotenv()

True

In [3]:
# LLM Define
llm = ChatOpenAI()

In [4]:
# state Define
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [5]:
# Function 1 

def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

# Function 2

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
# Graph Define
graph = StateGraph(JokeState)

#Node
graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

#Edges
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

# Compile the Graph
workflow = graph.compile(checkpointer=checkpointer)

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

In [ ]:
# To get Entire Store Things | History
workflow.get_state(config1)

In [ ]:
# To get Intermediate states value (for every node)
list(workflow.get_state_history(config1))

# Benfits of Persistance
we can implement Short term memory in chatbot.

fault tolerance. : resume chat exactly where crashes happen

HITL implement (Human in the loop).

Time Travel.

# FAULT TOLERANCE IMPLEMENTATION

In [2]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [3]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [4]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Delay introduced | Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [ ]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)


builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:

try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

OUTPUT
▶️ Running graph: Please manually interrupt during Step 2...

✅ Step 1 executed

⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)

In [ ]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})  # NONE    
# { None } Indicate : resume execution where it stop and providing same thread id of it.
print("\n✅ Final State:", final_state)

OUTPUT

✅ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)

✅ Step 3 executed

📌 Final State:

{
  
  'input': 'start',
  
  'step1': 'done',
  
  'step2': 'done',
  
  'step3': 'done'
}


In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

# HUMAN IN LOOP

(see by click on edit button)

topic --------> Linkedin Post -------> API
                                 ^
                                 |
                                 |
                                 |
persistance   ..............   Interrupt  ........... Here for
 
human interaction click the button to post on linkedin.

# Time Travel

After the Complete Execution we can go at any place|Node|checkpoint to see working after that or Replay the execution

What benfit? 

Debugging ( we can see where at which node | checkpoint error is occuring)



In [ ]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})
# we also give (checkpoint id) at where we want to resume execution

OUTPUT

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}, 'thread_id': '1'}, created_at='2025-07-29T21:56:38.565188+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f06cc6e-7230-65a8-bfff-0a96c2fc4e11'}}, tasks=(PregelTask(id='dcd96e38-1f32-5ed6-9f44-fa2b22c193f0', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza go to the doctor? Because it was feeling a little saucy!'}),), interrupts=())

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

OUTPUT

{'topic': 'pizza',
 'joke': 'Why did the mushroom go to the pizza party? Because he was a fungi and everyone wanted a pizza him!',
 'explanation': 'This joke plays on the word "fun guy" (fungi) which sounds like "fungi," a type of mushroom. The play on words is that the mushroom went to the pizza party because he was a "fun guy" and people wanted to "pizza" (see) him. The joke is a pun that combines the idea of mushrooms being fungi with the concept of being a fun person at a party.

# Updating state

we can also change the value of state at particular checkpoint 

like in previous conversation joke was on {Pizza}

now we can change topic (update state) : generate a joke on apple

In [ ]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5", "checkpoint_ns": ""}}, {'topic':'samosa'})

In [ ]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc72-ca16-6359-8001-7eea05e07dd2"}})